In [28]:
!pip install config

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import cv2
import numpy as np
import csv
import pickle
import random

from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [0]:
file_path = 'drive/My Drive/UGP/data/'
image_path = file_path + 'images/'
pkl_file = file_path + 'control_throttle.pkl'

In [31]:

with open(pkl_file, 'rb') as handle:
    samples = pickle.load(handle)

samples_list = [ [k, v[0], v[1], v[2]] for k, v in samples.items() ]

print (samples_list)
# print(type(samples))

[['img_1.png', 0.18930064571337885, 0.25, -4.800533255028889e-06], ['img_2.png', 0.396976893475627, 0.25, -4.2435166758552675e-06], ['img_3.png', 0.60368354465193, 0.25, -3.777350778423358e-06], ['img_4.png', 0.8093614980689562, 0.25, -3.3754453699405044e-06], ['img_5.png', 1.0140188635513085, 0.25, -3.021757634615436e-06], ['img_6.png', 1.2176726858615656, 0.25, -2.705674598228184e-06], ['img_7.png', 1.4203381036342928, 0.25, -2.4197537080364587e-06], ['img_8.png', 1.6220264262506094, 0.25, -2.1586106493767687e-06], ['img_9.png', 1.8227455931932444, 0.25, -1.918328881888123e-06], ['img_10.png', 2.0225011690937817, 0.25, -1.6960889499219886e-06], ['img_11.png', 2.2212972530015884, 0.25, -1.4898916859988028e-06], ['img_12.png', 2.4191371452144748, 0.25, -1.2983341810642124e-06], ['img_13.png', 2.6160238108369085, 0.25, -1.1204305435543632e-06], ['img_14.png', 2.81196022198145, 0.25, -9.554758821557998e-07], ['img_15.png', 3.006949615744835, 0.25, -8.029485706824968e-07], ['img_16.png', 

In [0]:
# Step2: Divide the data into training set and validation set
l = len(samples_list)
train_len = int(0.8*l)
valid_len = l - train_len

random.shuffle(samples_list)

train_samples = samples_list[:train_len]
validation_samples = samples_list[train_len:]

# train_samples, validation_samples = data.random_split(samples_list, [train_len, valid_len])

In [0]:
# Step3a: Define the augmentation, transformation processes, parameters and dataset for dataloader
def augment(imgName, angle):
  name = image_path + imgName
  current_image = cv2.imread(name)
  # print(type(current_image))
  # print(current_image.shape)
  # h = 400, w = 600, colors = 3
  current_image = current_image[65:-25, :, :]
  # h = 310, w = 600, colors = 3
  # print(type(current_image))
  # print(current_image.shape)
  if np.random.rand() < 0.5:
    current_image = cv2.flip(current_image, 1)
    angle = angle * -1.0  
  return current_image, angle

In [0]:
class Dataset(data.Dataset):

    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __getitem__(self, index):
        batch_samples = self.samples[index]
        steering_angle = float(batch_samples[3])
        center_img, steering_angle_center = augment(batch_samples[0], steering_angle)
        # left_img, steering_angle_left = augment(batch_samples[1], steering_angle + 0.4)
        # right_img, steering_angle_right = augment(batch_samples[2], steering_angle - 0.4)
        center_img = self.transform(center_img)
        # left_img = self.transform(left_img)
        # right_img = self.transform(right_img)
        return (center_img, batch_samples[1] , batch_samples[2] , steering_angle_center)
      
    def __len__(self):
        return len(self.samples)
                                  

In [0]:
# Step3b: Creating generator using the dataloader to parallasize the process
transformations = transforms.Compose([transforms.Lambda(lambda x: (x / 255.0) - 0.5)])

params = {'batch_size': 32,
          'shuffle': True,
          'num_workers': 4}

training_set = Dataset(train_samples, transformations)
training_generator = DataLoader(training_set, **params)


# validation_samples[0:32] is written to prevent error, since batch_size is 32
# CHANGE it later on when having more dataset
validation_set = Dataset(validation_samples[0:32], transformations)
validation_generator = DataLoader(validation_set, **params)

In [0]:
print(len(training_set))
print("training tuple size = "+str(len(training_set[0])))
print("vel = "+str(training_set[0][1]))
print("throttle = "+str(training_set[0][2]))
print("steering angle = "+str(training_set[0][3]))
print("Image dimension = "+str(len(training_set[0][0]))+" x "+str(len(training_set[0][0][0]))+" x "+str(len(training_set[0][0][0][0])))

print(len(validation_set))

160
training tuple size = 4
vel = 11.97780549912842
throttle = 0.1
steering angle = -5.093335655211608e-08
Image dimension = 310 x 600 x 3
32


In [0]:
class NetworkLight(nn.Module):

    def __init__(self):
        super(NetworkLight, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 24, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(24, 48, 5, stride=2),
            nn.MaxPool2d(4, stride=4),
            nn.Dropout(p=0.25)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=48*18*36 + 1, out_features=50),
            nn.ELU(),
            nn.Linear(in_features=50, out_features=10),
            nn.Linear(in_features=10, out_features=2)
        )
        

    def forward(self, input, vel):
        input = input.view(input.size(0), 3, 310, 600)
        output = self.conv_layers(input)
        
        # Append velocity in the output vector
        output = output.view(output.size(0), -1)
        vel = vel.view(vel.size(0),-1)
        # print(vel.shape)
        # print(output.shape)
        output = torch.cat((output,vel),dim = 1)
        output = self.linear_layers(output)
        return output

In [0]:
model = NetworkLight().float()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

criterion = nn.MSELoss()

# Step6: Check the device and define function to move tensors to that device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print('device is: ', device)

def toDevice(datas, device):
  
  imgs, vel, throttle, angles = datas
  return imgs.float().to(device), vel.float().to(device), throttle.float().to(device), angles.float().to(device)

device is:  cpu


In [0]:
def validation(model, validation_generator):
  print("")
  print("==========  Validation  ==========")
  print("")
  model.eval()
  valid_loss = 0
  with torch.set_grad_enabled(False):
      for local_batch, data in enumerate(validation_generator):
          # Transfer to GPU
          data = toDevice(data, device)
      
          # Model computations
          # optimizer.zero_grad()
          imgs, vel, throttle, angles = data
          with torch.no_grad():
            outputs = model(imgs,vel)
          
          throttle = torch.unsqueeze(throttle, 1)
          angles = torch.unsqueeze(angles,1)

          out_label = torch.cat([throttle,angles],dim=1)

          loss = criterion(output, out_label)
          
          valid_loss += loss.data.item()

          if local_batch % 16 == 0:
              print('Valid Loss: %.5f '
                  % (valid_loss/(local_batch+1)))
  print("")

In [0]:
max_epochs = 5
for epoch in range(max_epochs):
    print("")
    print("==========  Epoch %d  ==========" %epoch)
    model.to(device)
    
    # Training
    train_loss = 0
    model.train()
    for local_batch, data in enumerate(training_generator):
        # Transfer to GPU
        data = toDevice(data, device)
        
        # Model computations
        optimizer.zero_grad()
        model.zero_grad()
        model.train()
        imgs, vel, throttle, angles = data

        output = model(imgs,vel)

        throttle = torch.unsqueeze(throttle, 1)
        angles = torch.unsqueeze(angles,1)

        out_label = torch.cat([throttle,angles],dim=1)

        loss = criterion(output, out_label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        train_loss += loss.data.item()
            
        if local_batch % 40 == 0:
            print('Loss: %.5f '
                % (train_loss/(local_batch+1)))
    validation(model, validation_generator)
 


==========  Epoch 0  ==========
Loss: 0.27511 

==========  Validation  ==========

Valid Loss: 0.03725 


==========  Epoch 1  ==========
Loss: 0.02653 

==========  Validation  ==========

Valid Loss: 0.02473 


==========  Epoch 2  ==========
Loss: 0.02065 

==========  Validation  ==========

Valid Loss: 0.02607 


==========  Epoch 3  ==========
Loss: 0.01555 

==========  Validation  ==========

Valid Loss: 0.02752 


==========  Epoch 4  ==========
Loss: 0.01326 

==========  Validation  ==========

Valid Loss: 0.01783 



In [0]:
# Step8: Define state and save the model wrt to state
state = {
        'model': model.module if device == 'cuda' else model,
        }

torch.save(state, file_path + 'model.h5')

#Model saved

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type NetworkLight. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [40]:
eval_model = NetworkLight()

eval_state = torch.load(file_path + 'model.h5')

eval_model = eval_state['model']

eval_model.float()

eval_model.eval()

NetworkLight(
  (conv_layers): Sequential(
    (0): Conv2d(3, 24, kernel_size=(5, 5), stride=(2, 2))
    (1): ELU(alpha=1.0)
    (2): Conv2d(24, 48, kernel_size=(5, 5), stride=(2, 2))
    (3): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.25, inplace=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=31105, out_features=50, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=50, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=2, bias=True)
  )
)

In [0]:
def read(name):
  # print(name)
  current_image = cv2.imread(name)
  # h = 400, w = 600, colors = 3
  current_image = current_image[65:-25, :, :]
  # h = 310, w = 600, colors = 3
  return current_image

In [47]:
# params = {'batch_size': 1,
#           'shuffle': True,
#           'num_workers': 1}

transformations = transforms.Compose([transforms.Lambda(lambda x: (x / 255.0) - 0.5)])

file_path = 'drive/My Drive/UGP/data/'
image_path = file_path + 'images/'

class Dataset2(data.Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __getitem__(self, index):
        batch_samples = self.samples[index]
        img_name = image_path + batch_samples[0]
        # print(img_name)
        center_img = read(img_name)
        center_img = self.transform(center_img)
        return (center_img, batch_samples[1])
      
    def __len__(self):
        return len(self.samples)
  
print(samples_list[0])
test_set = Dataset2([samples_list[0]], transformations)
# print(test_set.samples)
test_generator = DataLoader(test_set, **params)

img, vel = test_set.__getitem__(0)
print(vel)
# print(test_generator)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print('device is: ', device)

def toDevice(datas, device):
  imgs, vel = datas
  return imgs.float().to(device), vel.float().to(device)

def testing(model, test_generator):
  model.eval()
  with torch.set_grad_enabled(False):
      for local_batch, data in enumerate(test_generator):
          # Transfer to GPU
          # print(local_batch)
          # print(data)
          data = toDevice(data, device)
          # print(data)
          imgs, vel = data
          with torch.no_grad():
            outputs = model(imgs,vel)
  return outputs

print(len(test_generator))
# for ind,data in enumerate(test_generator):
#   print(ind)
Result = testing(eval_model, test_generator)
# print(len(test_generator))

# test_image_tensor, test_vel_tensor = 
# Result = eval_model(test_image_tensor, test_vel_tensor)
print(Result)

['img_11.png', 2.2212972530015884, 0.25, -1.4898916859988028e-06]
2.2212972530015884
device is:  cpu
1
tensor([[ 0.2784, -0.0052]])


In [0]:
test_sample = samples_list[0]
transformations = transforms.Compose([transforms.Lambda(lambda x: (x / 255.0) - 0.5)])

test_image = read(file_path + 'images/img_1.png')

test_image_tensor = torch.tensor([test_image], requires_grad=False, dtype=torch.double)
# test_image_tensor = torch.tensor([test_image])
test_vel_tensor = torch.tensor(test_sample[1], requires_grad=False, dtype=torch.double)
# test_vel_tensor = torch.tensor(test_sample[1])

output = eval_model(test_image_tensor, test_vel_tensor)
